# Isotank Design for the LV4 Sub-Orbital Rocket
## Motivation
With liquid oxygen (LOX) being one of the most common oxidizers used in rocketry, lightweight cryogenic storage are a critical design component. Traditionally metallic materials, such as stainless steel, aluminimun and aluminum-lithium alloys have been partially due to their compatibility with LOX. Composite materials have also been looked at over the years, but finding specific materials that can withstand both the cold temperatures and remain non-reactive can be difficult. As part of the PSAS LV4 program, a capstone team at PSU built a small-scale composite tank design, but there are concerns with scaling up the design to LV4 scales.

In light of the difficulties manufacturing previous designs, a new approach for LV4 LOX tanks using aluminum isogrids is being studied. The goal is to have a design that is manufacturable by the amatuer group, but has a mass close to composite designs.

## Design Choices
### Material
For simplicity, the tank is built from a tube or pipe, eliminating the need to weld the length of the tank. The downside is that this limits the material options to what can be obtained off the shelf. A quick survey of choices found the following options, all made from 6061-T6. Note, material prices as of 2020/10

| ID (in) | Wall (in) | OD (in) | Type | Price/Ft | Source |
|---------|-----------|---------|------|----------|--------|
| 10.02 | 0.365 | 10.75 | Pipe | 270.01 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/10-nom-schedule-40-aluminum-pipe-6061-t6-extruded/pid/10619)|
| 10.02 | 0.365 | 10.75 | Pipe | 64.33 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|
| 10.50 | 0.750 | 12.00 | Tube | 826.71 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-75-wall-x-10-5-id-aluminum-round-tube-6061-t6-extruded-aluminum-association-in-/pid/21986)|
| 11.00 | 0.500 | 12.00 | Tube | 678.12 | [Metals Depot](https://www.metalsdepot.com/search?q=T3R12500)
| 11.50 | 0.250 | 12.00 | Tube | 181.63 | [Online Metals](https://www.onlinemetals.com/en/buy/aluminum/12-od-x-0-25-wall-x-11-5-id-aluminum-round-tube-6061-t6-extruded/pid/23929)|
| 12.00 | 0.375 | 12.75 | Pipe | 85.10 | [Midwest Steel & Al*](https://www.midweststeelsupply.com/store/6061aluminumpipe)|

*For Midwest Steel and Al, material is considered raw and can have light scratches

Other sizes appear to be available from [Vita Needle](https://www.vitaneedle.com/aluminum-tube/) but quotes have to be requested.

6061-T6 is a an acceptable choice as it has reasonable strength, weldability and LOX compatibility. The 11.5 in inner diameter (ID) pipe was selected as it's closest to the initial LV4 multi-disciplinary design targets and least expensive option.

For initial estimates, a total tank length of 48" is assumed. This fits with available manufacuring possibilities.

In [1]:
# Tank radial dimensions
tank_id = 11.5     # in
tank_wall_t = 0.25  # in
tank_od = tank_id+2*tank_wall_t # in
tank_r = tank_od/2   # in
tank_overall_length = 48  # in

# Material properties for Al 6061-T6
E = 69.0 # Young's Modules, GPa

E_imp = 10000 # ksi
F_ty = 40.0 # Yield Tensile Strength - ksi
F_tu = 45.0 # Ultimate Tensile Strength - ksi

nu = 0.33 # Poisson's Ratio


### Safety Factor
Minimum safety factor for Base 11 is 1.5 (see requirement 3.2.4.15)

In [2]:
safety_factor = 1.5

### Flanges
The use of the flange can increase the strength of the isogrid, but increases machining complexity. It was decided to omit flanges for initil version of the design.


In [3]:
flanges = False

### Skin Thickness
The tank stock material is not highly precise. The part will be machined from the outside in and there is a risk of inadvertantly maching through the pipe if the skin thickness is too small. Additionally, after talking with OMIC, they thought that a skin thickness of 0.05" (50 mil) was reasonable.

In [4]:
skin_t = 0.05


### Rib Depth
The isogrid ribs are the source of the strength of the isogrid design. The depth of the rib is the primary factor is determining the strength. Thefore, for optimimum strength, the rib depth is simply the wall thickness minus the skin thickness 

In [5]:
rib_depth = tank_wall_t - skin_t - 0.05
print('Rib Depth: {:5.3f}'.format(rib_depth))

Rib Depth: 0.150


### Rib thickness
Again, after discussion with OMIC, it was decided that a rib web thickness of 0.050" (50 mil) could be reasonably achievable.

In [6]:
rib_t = 0.05


### Pattern size
For optimal strength and to maintain isotropic properties, the isogrid cells need to wrap around the tank diameter in integer increments. Twenty four cells was chosen as a starting point.

In [7]:
import math
num_radial_divisions = 24
pattern_size = tank_od*math.pi/num_radial_divisions
print('Pattern Size: {:5.3f}'.format(pattern_size))

Pattern Size: 1.571


## Tank Properties
The isogrid properties are calculated from NASA CR-124075

In [8]:
cell_height = pattern_size
alpha = (rib_t*rib_depth)/(skin_t*cell_height) # Web non-dimensional ratio, pg. 2.0.008
t_eff = skin_t*(1+alpha) # Effective thickness 
delta = rib_depth/skin_t # rib depth to skin thickness ratio, pg. 2.0.008
if flanges == False:
    beta = (3*alpha*(1+delta)**2+(1+alpha)*(1+alpha*delta**2))**0.5
    t_star = skin_t*beta/(1+alpha) # equivilent thickness, Eq. 2.5.3
    t_bar = skin_t*(1+ 3*alpha)
    E_star = E*(1+alpha)**2.0/beta # equivilent Young's Modulus Eq. 2.5.4
print('Web non-dimensional ratio, Alpha: {:5.3f}'.format(alpha))
print('Rib depth to skin thickness, Delta: {:5.3f}'.format(delta))
print('Beta: {:5.3f}'.format(beta))
print('Skin thickness, t: {:5.3f} in'.format(skin_t))
print('Effective thickness, t_eff: {:5.3f} in'.format(t_eff))
print('Equivalant monocoque thickness, t*: {:5.3f} in'.format(t_star))
print('Equivalant weight thickness, t_bar: {:5.3f} in'.format(t_bar))
print('Equivalant monocoque Young\'s Modulus, E*: {:5.3f} GPa'.format(E_star))

Web non-dimensional ratio, Alpha: 0.095
Rib depth to skin thickness, Delta: 3.000
Beta: 2.573
Skin thickness, t: 0.050 in
Effective thickness, t_eff: 0.055 in
Equivalant monocoque thickness, t*: 0.117 in
Equivalant weight thickness, t_bar: 0.064 in
Equivalant monocoque Young's Modulus, E*: 32.182 GPa


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

In [9]:
import math

# Ring mass estimate from solidworks model
ring_mass = 0.5316 #kg = 1.172 lbs
ring_circum_density = 0.5316/(math.pi*12.0) # 12.0 since 12 was the diameter for the mass estimate
ring_length = 1.0*0.0254

weld_gap_length = 1.0*0.0254

grid_length = 48*0.0254-2*weld_gap_length-2*ring_length
grid_t_bar = t_bar*0.0254

density = 2700 #kg/m3
t = tank_wall_t*0.0254
id = tank_id*0.0254
ring_mass = 2.0*ring_circum_density*id
weld_gap_mass = 2.0*density*weld_gap_length*(math.pi*((id/2.0 + t)**2-(id/2.0)**2))
tank_wall_mass = density*grid_length*(math.pi*((id/2.0 + grid_t_bar)**2-(id/2.0)**2))

tank_mass = ring_mass + weld_gap_mass + tank_wall_mass

print('tank_mass:',tank_mass, 'kg')

tank_mass: 5.374328428123179 kg


## Load Calculations



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario. The example covers:
* General Instability (N_cr_1)
* Skin Buckling (N_cr_2)
* Rib Crippling (N_cr_3)

In [19]:
import math

# Inputs
F = 1.573 # Klbf ~7000 N
M = 0.265 # K in = 30 Nm

# Material
E_star_imp = E_imp*(1+alpha)**2.0/beta

# Instability
c0 = 0.397    # 4.2.005
c1 = 10.2     # 4.2.006
c2 = 0.616    # 4.2.008

# Classical Value
N_cl = (1/(3*(1-nu*nu))**0.5)*E_star_imp/1000*t_star*t_star/tank_r # k/in

N_a = F/(2*math.pi*tank_r) # k/in
N_b = M/(math.pi*tank_r**2) # k/in
N_cr = N_a + N_b

N_cr_1 = c0*E_imp*skin_t*skin_t/tank_r*beta
N_cr_2 = c1*E_imp*skin_t*(1+alpha)*(skin_t*skin_t)/(pattern_size*pattern_size)
N_cr_3 = c2*E_imp*skin_t*(1+alpha)*(rib_t*rib_t)/(rib_depth*rib_depth)

p_burst = F_tu*1000*skin_t*(1+alpha)/tank_r
p_meop = p_burst/1.5

print('General Instability - ',N_cr_1,'k/in',N_cr_1*2*math.pi*tank_r,'klbf')
print('Skin Buckling       - ',N_cr_2,'k/in',N_cr_2*2*math.pi*tank_r,'klbf')
print('Rib Cripling        - ',N_cr_3,'k/in',N_cr_3*2*math.pi*tank_r,'klbf')

print('P_burst             - ',p_burst,'psi')
print('P_MEOP              - ',p_meop,'psi')


# Determine if burst or compression critical
# Fig 4.2.3 was reconstructed and curve fits were derived
# See fig_4.2.3_reconstruction for details
fig_x = N_cr/(E_imp*tank_r)*10**7
p_0 = 1.134*(fig_x**0.595)*F_tu
#t_m = 1.650*(fig_x**0.597)*tank_r
print()
print('P_0                 - ',p_0,'psi')
if(p_burst > p_0):
    print('P_burst > P_0\nDesign is Burst Critical')
else:
    print('P_0 > P_burst\nDesign is Compression Critical')

General Instability -  4.256277931192595 k/in 160.45789776325222 klbf
Skin Buckling       -  5.660828842587181 k/in 213.4082196612132 klbf
Rib Cripling        -  37.49020372037581 k/in 1413.3473830742084 klbf
P_burst             -  410.8098621956765 psi
P_MEOP              -  273.8732414637843 psi

P_0                 -  167.1399785773593 psi
P_burst > P_0
Design is Burst Critical


## Stress Calcs
x is the hoop coordinate, y is the longitudinal coordinate

In [11]:
Nx = p_meop*tank_r
Ny = p_meop*tank_r/2
Nxy = 0.0

sigma_x = Nx/t_eff
sigma_y = Ny/t_eff

sigma_1 = 1/(3*t_eff)*(3*Nx-Ny)
sigma_2 = 2/(3*t_eff)*(Ny+3**0.5*Nxy)
sigma_3 = 2/(3*t_eff)*(Ny-3**0.5*Nxy)

print('Nx:',Nx,'lbf/in')
print('Ny:',Ny,'lbf/in')
print('sigma_x:',sigma_x,'psi')
print('sigma_y:',sigma_y,'psi') 
print('sigma_1:',sigma_1,'psi')
print('sigma_2:',sigma_2,'psi')
print('sigma_3:',sigma_3,'psi')


Nx: 1643.239448782706 lbf/in
Ny: 821.619724391353 lbf/in
sigma_x: 29999.999999999996 psi
sigma_y: 14999.999999999998 psi
sigma_1: 24999.999999999996 psi
sigma_2: 9999.999999999998 psi
sigma_3: 9999.999999999998 psi


## Center Weld Strength and Safety Factor

In [12]:
import math

F_tu_welded = 27.0 # ksi
F_su_welded = 15.0 # ksi - 4043/4047 Filler Rod
#F_su_welded = 15.0 # ksi - 5554 Filler Rod

center_weld_depth = 0.25
area = math.pi*tank_od*center_weld_depth
print('weld area',area,'in2')
F_max_welded = F_tu_welded*area
print('Tensile force',F_max_welded,'klbf')
p_burst_center_weld = F_tu_welded*1000*center_weld_depth/tank_r
print('Center Weld Burst Pressure',p_burst_center_weld,'psi')
center_weld_safety_factor = p_burst_center_weld/p_meop
print('Center Weld Safety Factor',center_weld_safety_factor)


weld area 9.42477796076938 in2
Tensile force 254.46900494077323 klbf
Center Weld Burst Pressure 1125.0 psi
Center Weld Safety Factor 4.107739748458648


## End Cap Weld Strength and Safety Factor

In [13]:
import math

end_cap_weld_width = 0.25
F_ec_max = F_su_welded*end_cap_weld_width*2*math.pi*tank_r
print('Max Force on End Cap',F_ec_max,'klbf')
p_burst_end_cap = F_ec_max*1000/(math.pi*tank_r**2)
print('End Cap Weld Burst Pressure',p_burst_end_cap,'psi')
ec_weld_safety_factor = p_burst_end_cap/p_meop
print('End Cap Weld Safety Factor',ec_weld_safety_factor)


Max Force on End Cap 141.37166941154067 klbf
End Cap Weld Burst Pressure 1249.9999999999998 psi
End Cap Weld Safety Factor 4.564155276065164


## Thermal Expansion (Contraction)

In [14]:
cte = 13*10**-6 # in/in F
dt = -297-70 # Room temperature to liquid oxygen temp
delta_diameter = cte*tank_od*dt
print(delta_diameter)

-0.057252
